In [2]:
from sklearn.svm import SVR
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [3]:
df = datasets.load_diabetes(as_frame=True).frame
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [4]:
df.shape

(442, 11)

In [5]:
X = df.drop("target", axis=1)
y = df["target"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [7]:
y_scaler = StandardScaler()

y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1,1)).ravel()
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1,1)).ravel()

In [8]:
model = SVR()
model.fit(X_train, y_train_scaled)

SVR()

In [9]:
y_pred_scaled = model.predict(X_test)

In [10]:
r2_score(y_test_scaled, y_pred_scaled)

0.48844443151651895

In [11]:
model = SVR(kernel="linear")
model.fit(X_train, y_train_scaled)
y_pred_scaled = model.predict(X_test)
r2_score(y_test_scaled, y_pred_scaled)

0.44337613238337736

In [12]:
model = SVR(kernel="poly")
model.fit(X_train, y_train_scaled)
y_pred_scaled = model.predict(X_test)
r2_score(y_test_scaled, y_pred_scaled)

0.24203771038107835

In [13]:
model = SVR(kernel="sigmoid")
model.fit(X_train, y_train_scaled)
y_pred_scaled = model.predict(X_test)
r2_score(y_test_scaled, y_pred_scaled)

-15.316808189576822

HyperParamter Tuning using GridSearchCV

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
param_grid = {
    "C" : [1,2,5,10,20,50,100],
    "kernel" : ["rbf", "linear"],
    "epsilon" : [0.01, 0.1, 0.2, 0.3, 0.5]
}

In [16]:
svr = SVR()

grid_search = GridSearchCV(svr, param_grid, scoring="r2", cv=5)

grid_search.fit(X_train, y_train_scaled)


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 2, 5, 10, 20, 50, 100],
                         'epsilon': [0.01, 0.1, 0.2, 0.3, 0.5],
                         'kernel': ['rbf', 'linear']},
             scoring='r2')

In [17]:
print("best params - ", grid_search.best_params_)

best params -  {'C': 20, 'epsilon': 0.1, 'kernel': 'linear'}


In [18]:
best_model = SVR(kernel="linear", C=10, epsilon=0.1)
best_model.fit(X_train, y_train_scaled)
y_pred_scaled = best_model.predict(X_test)
r2_score(y_test_scaled, y_pred_scaled)

0.47444183250401084